In [1]:
import numpy as np
import math
import pandas as pd


In [10]:
class My_SVC:
    def __init__(self,dimensions,initial_weights = 'random',initial_bias ='random'):
        self.dimensions = dimensions
        if initial_weights == 'random':
            self.weights = []
            for i in range(self.dimensions):
                self.weights.append(np.random.random())   
            self.weights = np.array(self.weights)
        elif initial_weights == 'zeros':
            self.weights = []
            for i in range(self.dimensions):
                self.weights.append(0)
            self.weights = np.array(self.weights)
        else :
            self.weights = initial_weights.copy()
        if initial_bias == 'random':
            self.bias = np.random.random()
        elif initial_weights == 'zeros':
            self.bias = 0
        else :
            self.bias = initial_bias
        self.weights = np.array(self.weights)
    def predict(self,v):
        if len(v.shape) == 1:
            return self.weights.dot(v) + self.bias
        elif len(v.shape) == 2:
            return [self.weights.dot(vector)+self.bias for vector in v]
    def error(self,label,prediction):
        return (label - prediction)**2
    def _pd_error_Wm(self,v,label,prediction,m):
        return -2*(label-prediction)*v[m]
    def _pd_error_bias(self,label,prediction):
        return  -2*(prediction-label)
    def mean_error(self,labels,predictions):
        summation = 0
        for i,label in enumerate(labels):
            summation += self.error(label,predictions[i])
        return summation/(i+1)
    def pd_mean_error_Wm(self,V,labels,predictions,m):
        summation = 0
        for i,vector in enumerate(V):
            summation += self._pd_error_Wm(vector,labels[i],predictions[i],m)
        return summation/(i+1)
    def pd_mean_error_bias(self,labels,predictions):
        summation = 0
        for i,label in enumerate(labels):
            summation += self._pd_error_bias(label,predictions[i])
        return summation/(i+1)

    def train_gd(self,epochs,data,labels, learning_rate=0.1,threshold=0.1):
        if data.shape[1] != self.dimensions or len(data) != len(labels):
               return
        length = len(data)
        epoch = 0
        P = self.predict(data)
        E = self.mean_error(labels,P)
        
        while(epoch<epochs and E > threshold):
            d = 0
            while (d < self.dimensions):
                self.weights[d] -= learning_rate * self.pd_mean_error_Wm(data,labels,P,d)
                d += 1
            self.bias -= learning_rate * self.pd_mean_error_bias(labels,P)
            P = self.predict(data)
            E = self.mean_error(labels,P)
            print("Epoch :%d"%epoch)
            print("Weights : "+str(self.weights))
            print("Bias : "+ str(self.bias))
            print("Mean squared error:"+str(E))
            epoch += 1
    def train_sgd(self,epochs,data,labels, learning_rate=0.1,threshold=0.1):
            if data.shape[1] != self.dimensions or len(data) != len(labels):
                return
            length = len(data)
            epoch = 0
            P = self.predict(data)
            E = self.mean_error(labels,P)
            #print('yes')
            while(epoch<epochs and E > threshold):
                k = 0
                while(k<length):
                    d = 0
                    p = self.predict(data[k])
                    while (d < self.dimensions):
                        self.weights[d] -= learning_rate * self._pd_error_Wm(data[k],labels[k],p,d)
                        p = self.predict(data[k])
                        d += 1
                    self.bias -= learning_rate * self._pd_error_bias(labels[i],p)
                    k += 1

                P = self.predict(data)
                E = self.mean_error(labels,P)
                print("Epoch :%d"%epoch)
                print("Weights : "+str(self.weights))
                print("Bias : "+ str(self.bias))
                print("Mean squared error:"+str(E))
                epoch += 1
    def train_sgd2(self,epochs,data,labels, learning_rate=0.1,threshold=0.1):
        if data.shape[1] != self.dimensions or len(data) != len(labels):
            return
        length = len(data)
        epoch = 0
        P = self.predict(data)
        E = self.mean_error(labels,P)
        #print('yes')
        while(epoch<epochs and E > threshold):
            d = 0
            while(d < self.dimensions):
                k = 0
                while (k<length):
                    p = self.predict(data[k])
                    self.weights[d] -= learning_rate * self._pd_error_Wm(data[k],labels[k],p,d)
                    k += 1
                
                d += 1
            k = 0
            while (k<length):
                p = self.predict(data[k])
                self.bias -= learning_rate * self._pd_error_bias(labels[i],p)
                k += 1
               
            P = self.predict(data)
            E = self.mean_error(labels,P)
            print("Epoch :%d"%epoch)
            print("Weights : "+str(self.weights))
            print("Bias : "+ str(self.bias))
            print("Mean squared error:"+str(E))
            epoch += 1

Weights : [-0.11287131  0.17393038  0.00820932  0.00018424]
Bias : 0.6319545089960352
Mean squared error:3.475381168702721

In [4]:
df = pd.read_csv('E:\\insurance.csv')
print(df)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
5      31  female  25.740         0     no  southeast   3756.62160
6      46  female  33.440         1     no  southeast   8240.58960
7      37  female  27.740         3     no  northwest   7281.50560
8      37    male  29.830         2     no  northeast   6406.41070
9      60  female  25.840         0     no  northwest  28923.13692
10     25    male  26.220         0     no  northeast   2721.32080
11     62  female  26.290         0    yes  southeast  27808.72510
12     23    male  34.400         0     no  southwest   1826.84300
13     56  female  39.820         0     no  southeast  11090.7

In [5]:
V = []
C = []

for i in range(df.shape[0]):
    v = []
    v.append(df['age'][i])
    if (df['sex'][i]=='male'):
        v.append(1)
    else:
        v.append(-1)
    v.append(df['bmi'][i])
    v.append(df['charges'][i])
    V.append(v)
    if (df['smoker'][i]=='yes'):
        C.append(1)
    else:
        C.append(-1) 
V = np.array(V)
C = np.array(C)
length = len(V)
dimensions = V.shape[1]

In [6]:
learning_rate = 0.1

In [18]:
svc = My_SVC(dimensions)

In [19]:
svc.train_sgd2(10000,V,C,learning_rate = 0.000000001005)


Epoch :0
Weights : [-9.20836608e-02  8.92861304e-01  4.36185088e-01  4.14173550e-04]
Bias : 0.7137398667025404
Mean squared error:304.92943254782836
Epoch :1
Weights : [-9.38460239e-02  8.92857426e-01  4.34767669e-01  4.15433438e-04]
Bias : 0.7137799064864552
Mean squared error:301.81701767010384
Epoch :2
Weights : [-9.55975126e-02  8.92853548e-01  4.33358240e-01  4.16685265e-04]
Bias : 0.7138196901435071
Mean squared error:298.742409342254
Epoch :3
Weights : [-9.73381911e-02  8.92849670e-01  4.31956755e-01  4.17929077e-04]
Bias : 0.7138592191819396
Mean squared error:295.70515415120224
Epoch :4
Weights : [-9.90681230e-02  8.92845792e-01  4.30563167e-01  4.19164924e-04]
Bias : 0.7138984951011078
Mean squared error:292.70480406859025
Epoch :5
Weights : [-1.00787372e-01  8.92841914e-01  4.29177429e-01  4.20392850e-04]
Bias : 0.7139375193915355
Mean squared error:289.7409163871541
Epoch :6
Weights : [-1.02496001e-01  8.92838036e-01  4.27799496e-01  4.21612904e-04]
Bias : 0.713976293534960

Epoch :55
Weights : [-1.74427727e-01  8.92648114e-01  3.68949054e-01  4.72655955e-04]
Bias : 0.7155984174589739
Mean squared error:179.62926615014376
Epoch :56
Weights : [-1.75678124e-01  8.92644240e-01  3.67908061e-01  4.73536393e-04]
Bias : 0.7156263936659922
Mean squared error:178.0448341363436
Epoch :57
Weights : [-1.76920671e-01  8.92640367e-01  3.66872845e-01  4.74411013e-04]
Bias : 0.7156541847754481
Mean squared error:176.479825887894
Epoch :58
Weights : [-1.78155413e-01  8.92636494e-01  3.65843372e-01  4.75279851e-04]
Bias : 0.715681791877005
Mean squared error:174.93400665940354
Epoch :59
Weights : [-1.79382396e-01  8.92632621e-01  3.64819608e-01  4.76142940e-04]
Bias : 0.715709216053901
Mean squared error:173.40714450435726
Epoch :60
Weights : [-1.80601667e-01  8.92628748e-01  3.63801520e-01  4.77000315e-04]
Bias : 0.7157364583829926
Mean squared error:171.89901024198304
Epoch :61
Weights : [-1.81813271e-01  8.92624875e-01  3.62789074e-01  4.77852009e-04]
Bias : 0.7157635199

Epoch :112
Weights : [-2.34425813e-01  8.92427548e-01  3.17913179e-01  5.14488776e-04]
Bias : 0.7169271431732548
Mean squared error:114.42820898649401
Epoch :113
Weights : [-2.35295331e-01  8.92423683e-01  3.17152690e-01  5.15087094e-04]
Bias : 0.7169461340859813
Mean squared error:113.64656667203145
Epoch :114
Weights : [-2.36159249e-01  8.92419819e-01  3.16396330e-01  5.15681266e-04]
Bias : 0.7169649927875846
Mean squared error:112.87460199949585
Epoch :115
Weights : [-2.37017601e-01  8.92415954e-01  3.15644074e-01  5.16271317e-04]
Bias : 0.7169837200560626
Mean squared error:112.11219669793408
Epoch :116
Weights : [-2.37870419e-01  8.92412090e-01  3.14895898e-01  5.16857270e-04]
Bias : 0.71700231666483
Mean squared error:111.3592339146374
Epoch :117
Weights : [-2.38717735e-01  8.92408225e-01  3.14151779e-01  5.17439151e-04]
Bias : 0.7170207833827431
Mean squared error:110.61559819830613
Epoch :118
Weights : [-2.39559584e-01  8.92404361e-01  3.13411692e-01  5.18016983e-04]
Bias : 0.7

Epoch :168
Weights : [-2.75345474e-01  8.92211406e-01  2.81061235e-01  5.42240866e-04]
Bias : 0.7178070095928044
Mean squared error:82.7438719604392
Epoch :169
Weights : [-2.75947383e-01  8.92207552e-01  2.80498280e-01  5.42641136e-04]
Bias : 0.7178196770561925
Mean squared error:82.35585875776381
Epoch :170
Weights : [-2.76545277e-01  8.92203698e-01  2.79938294e-01  5.43038435e-04]
Bias : 0.717832249509779
Mean squared error:81.97268226172658
Epoch :171
Weights : [-2.77139177e-01  8.92199845e-01  2.79381260e-01  5.43432781e-04]
Bias : 0.7178447275123354
Mean squared error:81.59428243912082
Epoch :172
Weights : [-2.77729109e-01  8.92195992e-01  2.78827161e-01  5.43824191e-04]
Bias : 0.7178571116193421
Mean squared error:81.22059998235663
Epoch :173
Weights : [-2.78315094e-01  8.92192139e-01  2.78275978e-01  5.44212684e-04]
Bias : 0.7178694023830098
Mean squared error:80.85157630080604
Epoch :174
Weights : [-2.78897158e-01  8.92188286e-01  2.77727696e-01  5.44598275e-04]
Bias : 0.717881

Epoch :225
Weights : [-3.03887776e-01  8.91992096e-01  2.53240204e-01  5.60792744e-04]
Bias : 0.7183925456713449
Mean squared error:66.84375608497712
Epoch :226
Weights : [-3.04294907e-01  8.91988255e-01  2.52821476e-01  5.61049036e-04]
Bias : 0.7184006002419853
Mean squared error:66.65382517557441
Epoch :227
Weights : [-3.04699175e-01  8.91984414e-01  2.52404872e-01  5.61303214e-04]
Bias : 0.718408586922963
Mean squared error:66.46624992407178
Epoch :228
Weights : [-3.05100598e-01  8.91980574e-01  2.51990380e-01  5.61555291e-04]
Bias : 0.7184165061132403
Mean squared error:66.28100042474978
Epoch :229
Weights : [-3.05499193e-01  8.91976734e-01  2.51577987e-01  5.61805278e-04]
Bias : 0.718424358209423
Mean squared error:66.09804713751564
Epoch :230
Weights : [-3.05894975e-01  8.91972894e-01  2.51167681e-01  5.62053190e-04]
Bias : 0.7184321436057867
Mean squared error:65.91736088352414
Epoch :231
Weights : [-3.06287963e-01  8.91969055e-01  2.50759449e-01  5.62299037e-04]
Bias : 0.718439

...

Epoch :3597
Weights : [-1.96733676e-01  8.80434418e-01  5.14611627e-02  3.74435422e-04]
Bias : 0.7078814879702762
Mean squared error:17.21241056227114
Epoch :3598
Weights : [-1.96696836e-01  8.80431319e-01  5.14338874e-02  3.74387361e-04]
Bias : 0.7078775900098014
Mean squared error:17.20695491596603
Epoch :3599
Weights : [-1.96660005e-01  8.80428221e-01  5.14066186e-02  3.74339312e-04]
Bias : 0.7078736919587872
Mean squared error:17.201501508233267
Epoch :3600
Weights : [-1.96623182e-01  8.80425122e-01  5.13793563e-02  3.74291275e-04]
Bias : 0.7078697938172527
Mean squared error:17.196050338094075
Epoch :3601
Weights : [-1.96586368e-01  8.80422024e-01  5.13521005e-02  3.74243248e-04]
Bias : 0.7078658955852182
Mean squared error:17.19060140456993
Epoch :3602
Weights : [-1.96549563e-01  8.80418926e-01  5.13248512e-02  3.74195234e-04]
Bias : 0.7078619972627075
Mean squared error:17.185154706682702
Epoch :3603
Weights : [-1.96512766e-01  8.80415828e-01  5.12976083e-02  3.74147230e-04]
Bia

Epoch :3654
Weights : [-1.94647643e-01  8.80258032e-01  4.99167856e-02  3.71714038e-04]
Bias : 0.7076591603441094
Mean squared error:16.904983015380243
Epoch :3655
Weights : [-1.94611297e-01  8.80254942e-01  4.98898778e-02  3.71666621e-04]
Bias : 0.707655257256798
Mean squared error:16.89965341823674
Epoch :3656
Weights : [-1.94574959e-01  8.80251851e-01  4.98629765e-02  3.71619215e-04]
Bias : 0.7076513540801572
Mean squared error:16.894326004569244
Epoch :3657
Weights : [-1.94538629e-01  8.80248762e-01  4.98360815e-02  3.71571821e-04]
Bias : 0.707647450814207
Mean squared error:16.889000773423863
Epoch :3658
Weights : [-1.94502309e-01  8.80245672e-01  4.98091930e-02  3.71524437e-04]
Bias : 0.7076435474589717
Mean squared error:16.883677723847065
Epoch :3659
Weights : [-1.94465997e-01  8.80242582e-01  4.97823108e-02  3.71477065e-04]
Bias : 0.7076396440144658
Mean squared error:16.878356854885805
Epoch :3660
Weights : [-1.94429693e-01  8.80239493e-01  4.97554351e-02  3.71429704e-04]
Bia

Epoch :3711
Weights : [-1.92589559e-01  8.80082123e-01  4.83932193e-02  3.69029083e-04]
Bias : 0.7074365424118166
Mean squared error:16.60465306377498
Epoch :3712
Weights : [-1.92553699e-01  8.80079041e-01  4.83666742e-02  3.68982301e-04]
Bias : 0.7074326342662076
Mean squared error:16.599446414860772
Epoch :3713
Weights : [-1.92517848e-01  8.80075959e-01  4.83401353e-02  3.68935529e-04]
Bias : 0.707428726032464
Mean squared error:16.594241895732896
Epoch :3714
Weights : [-1.92482005e-01  8.80072877e-01  4.83136028e-02  3.68888769e-04]
Bias : 0.7074248177106048
Mean squared error:16.589039505461635
Epoch :3715
Weights : [-1.92446171e-01  8.80069796e-01  4.82870766e-02  3.68842020e-04]
Bias : 0.7074209093006506
Mean squared error:16.58383924311797
Epoch :3716
Weights : [-1.92410346e-01  8.80066715e-01  4.82605567e-02  3.68795282e-04]
Bias : 0.7074170008026219
Mean squared error:16.57864110777305
Epoch :3717
Weights : [-1.92374528e-01  8.80063633e-01  4.82340431e-02  3.68748555e-04]
Bias

Epoch :3766
Weights : [-1.90629831e-01  8.79912832e-01  4.69425784e-02  3.66472413e-04]
Bias : 0.7072214640657665
Mean squared error:16.32142586138605
Epoch :3767
Weights : [-1.90594435e-01  8.79909758e-01  4.69163785e-02  3.66426235e-04]
Bias : 0.707217551103455
Mean squared error:16.316334981266213
Epoch :3768
Weights : [-1.90559048e-01  8.79906684e-01  4.68901848e-02  3.66380068e-04]
Bias : 0.7072136380541458
Mean squared error:16.31124618042597
Epoch :3769
Weights : [-1.90523669e-01  8.79903611e-01  4.68639974e-02  3.66333912e-04]
Bias : 0.707209724917857
Mean squared error:16.306159457958586
Epoch :3770
Weights : [-1.90488298e-01  8.79900537e-01  4.68378162e-02  3.66287767e-04]
Bias : 0.7072058116946098
Mean squared error:16.30107481295775
Epoch :3771
Weights : [-1.90452935e-01  8.79897464e-01  4.68116413e-02  3.66241632e-04]
Bias : 0.7072018983844269
Mean squared error:16.295992244517706


KeyboardInterrupt: 

In [71]:
for i in C:
    print('yes')
    break

yes


In [212]:
svc.weights = np.array([-0.11287131 , 0.17393038 , 0.00820932 , 0.00018424])
svc.bias= 0.6319545089960352


In [20]:
P = svc.predict(V)
E = svc.mean_error(C,P)
print("Weights : "+str(svc.weights))
print("Bias : "+ str(svc.bias))
print("Mean squared error:"+str(E))

Weights : [-1.90436087e-01  8.79897464e-01  4.68116413e-02  3.66241632e-04]
Bias : 0.7072018983844269
Mean squared error:16.295548651489877


In [13]:
tw = svc.weights.copy()
tb = svc.bias

In [223]:
svc.weights = tw.copy()
svc.bias = tb

In [24]:
correct = 0
for i in range(len(P)):
    if C[i] == 1 :
        if P[i] > 0 :
            correct += 1
    if C[i] == -1 :
        if P[i] < 0:
            correct += 1
print(correct/len(P))

0.9013452914798207
